## Acute experiments with audio stim blocks, no comm with recording system
Uses raspmachine_pyaduio for the sbc state machine
Plays audio stim blocks
Doesn't pass metadata to recording system
Used mainly for acute experiments with the neuropixels on sglx and si-probes on intan rhd system.


In [1]:
import zmq
import time
import socket
import os
import sys
import logging
import threading
import warnings
import schedule
import numpy as np
from numpy import matlib

# setup the logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)

from rigmq.host import sbc

2019-07-11 17:24:05,889 root         INFO     Computer: lookfar


### Session tools and methods

In [2]:
# open the devices
#rasp_ip = '169.254.53.135' #raspi on direct connect on mac
rasp_ip = '192.168.1.50'
pi_zmq = sbc.SBC(port='5559', ip=rasp_ip) # raspi on direct connect on mac
pi_zmq.connect()

In [3]:
# make a rig dictionary
# soon to be object
rig_par = {'sbc': pi_zmq,
          'oe': None}

## very simple single trial tests

In [4]:
### commands to be sent to raspmachine_pyaudio.py
def cmd_trial_number(tr_num):
    #on is boolean
    return 'trial_number number {}'.format(int(tr_num)).encode('utf8')
        
def cmd_switch_glass(switch: str) -> str:
    return 'glass switch {}'.format(switch).encode('utf8')

# command functions
def cmd_play_wav(wav_file):
    cmd_line = 'play_wav stim_file {}'.format(wav_file).encode('utf8')
    return cmd_line

def cmd_audio_trial(wav_file):
    cmd_line = 'trial_audio stim_file {}'.format(wav_file).encode('utf8')
    return cmd_line

def cmd_stim_pulse():
    cmd_line = 'trial_stim_pulse'.encode('utf8')
    return cmd_line
    
# def cmd_trial_pin(on):
#     #on is boolean
#     return 'trial_pin on {}'.format(on).encode('utf8')

####

# this is dirty, the run_trial is the same for all 'types' of trial
# (stimpulse, audio, smartglass)
# There should be just a Trial class, with a method run().
def run_trial(rig_par={'sbc': None, 'oe':None}, 
              trial_par={'type': None, 'iti':120*1000., 'tr_num': 5, 'wave_file': None, 'transparent_time': 10*1000}):
    # intertrial interval
    time.sleep(int(trial_par['iti']*100)/100000.)
    
    # a very simple trial; switch on, wait, switch off
    rig_par['sbc'].send_command(cmd_trial_number(trial_par['tr_num']))
    
    if trial_par['type'] == 'smartglass':
        pi_zmq.send_command(cmd_switch_glass('on'))
        time.sleep(int(trial_par['transparent_time']*100)/100000.)
        pi_zmq.send_command(cmd_switch_glass('off'))
    
    if trial_par['type'] == 'audio':
        #rig_par['sbc'].send_command(cmd_trial_pin(True))
        rig_par['sbc'].send_command(cmd_audio_trial(trial_par['stim']))
        #rig_par['sbc'].send_command(cmd_trial_pin(False))
        
    if trial_par['type'] == 'stimpulse':
        #rig_par['sbc'].send_command(cmd_trial_pin(True))
        rig_par['sbc'].send_command(cmd_stim_pulse())
        #rig_par['sbc'].send_command(cmd_trial_pin(False))

        
class BlockGlass:
    # A block is a generator that gives trials (dictionaries)
    def __init__(self, transp_time_list, iti_bounds_ms, size, random = True):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = transp_time_list
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None, 'transparent_time': 10*1000}
        self.stim_order = matlib.repmat(np.arange(len(transp_time_list)),
                                           int(np.ceil(size/len(transp_time_list))),
                                           1)[:size].flatten()
        if random:
            np.random.shuffle(self.stim_order)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['type'] = 'smartglass'
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = None
            self.trial_pars['transparent_time'] = self.stim_list[self.stim_order[self.trials]]
            self.trials+=1
            return self.trial_pars
        else:
            raise StopIteration()
            
class BlockAudio:
    # A block is a generator that gives trials (dictionaries)
    def __init__(self, file_path_list, iti_bounds_ms, size, random = True):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = file_path_list
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None, 'transparent_time': None}
        self.stim_order = matlib.repmat(np.arange(len(self.stim_list)),
                                           int(np.ceil(size/len(self.stim_list))),
                                           1)[:size].flatten()
        if random:
            np.random.shuffle(self.stim_order)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['type'] = 'audio'
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = self.stim_list[self.stim_order[self.trials]]
            self.trial_pars['transparent_time'] = None
            self.trials += 1
            return self.trial_pars
        else:
            raise StopIteration()
            
class BlockStimPulse:
    # A block is a generator that gives trials (dictionaries)
    def __init__(self, iti_bounds_ms, size):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = None
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None, 'transparent_time': None}
        self.stim_order = np.arange(size)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['type'] = 'stimpulse'
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = None
            self.trial_pars['transparent_time'] = None
            self.trials += 1
            return self.trial_pars
        else:
            raise StopIteration()

### Create a block of trials that play audio files

In [17]:
#block_files = ['bos.wav', 'con.wav', 'rev.wav', 'syn.wav']
block_files = ['tone_test.wav']
block_iti_bounds = [5*1000, 6*1000] # it ranges from 5 to 15 secs, uniformly distributed
audio_block = BlockAudio(block_files, block_iti_bounds, 80) # block is 80 trial

In [6]:
audio_block.next()

{'iti': 5743,
 'tr_num': 0,
 'stim': 'tone_test.wav',
 'transparent_time': None,
 'type': 'audio'}

### Make a Runner Thread that goes through a block running each trial

In [8]:
class Runner(threading.Thread):
    # runs an experiment block
    # forced to no recording system to control
    def __init__(self, block, rig_par, recorder=None, group=None, target=None, name=None, 
                 args=(), kwargs=None):
        threading.Thread.__init__(self, group=group, target=target, name=name)
        
        logging.info('Initializing block')
        self.block = block #block object
        self.rig = rig_par
        self.stim_sys = rig_par['sbc']
        self.rec_sys = None
        self.running = threading.Event() #Event
        self.finished = False
        self.trial = None
        self.running.clear()
        self.recorder = recorder #Record control
   
    def run(self):
        logging.debug('running block')
        if not self.running.is_set():
            logger.info('Starting Block')
#             if self.is_recording()==True:
#                 logging.debug('Starting recording')
#                 self.recorder.start()
#                 time.sleep(2)
            self.running.set()
            while True:
                try:
                    if self.running.is_set():
                        self.trial = self.block.next()
                        logging.info('Running trial {}'.format(self.trial))
                        print(self.trial)
                        run_trial(rig_par=self.rig, trial_par=self.trial)
                    else:
                        self.end_block()
                        break

                except StopIteration:
                    self.finished = True
                    self.running.clear()
                    logger.info('finished block')
                    break
            
            if self.is_recording()==True:
                logger.info('Runner stopping recording')
                self.recorder.signal_stop()
        else:
            logging.info('Cant start, already running')
            
        return self.finished
                    
    def end_block(self):
        print('stopping the block in trial {}'.format(self.trial['tr_num']))
        self.finished = False
        
    def signal_stop(self):
        self.running.clear()
        
    def is_running(self):
        return self.running.is_set()
    
    def is_recording(self):
        logging.debug('runner checking recording status')
        logging.debug('{}'.format(self.recorder))
        if self.recorder is not None:
            logging.debug('status {}'.format(self.recorder.report_recording()))
            return self.recorder.report_recording()
        else:
            return None

#### Run a block till the end

In [33]:
all_files = ['20190701-f03123-bos-undirected',
 '20190705-f00201-bos-calls-undirected',
 '20190705-f00228-bos-directed',
 '20190705-f00287-bos-calls-directed',
 '20190705-f00287-calls',
 '20190705-f00254-bos-undirected']
all_files = ['{}_tag.wav'.format(i) for i in all_files]

#block_files = ['bos.wav', 'con.wav', 'rev.wav', 'syn.wav']
#block_files = ['tone_test.wav']

block_files = all_files
block_iti_bounds = [3*1000, 7*1000] # it ranges from 5 to 15 secs, uniformly distributed
audio_block = BlockAudio(block_files, block_iti_bounds, 120) # block is 80 trial

block_runner = Runner(audio_block, rig_par, recorder=None)
block_runner.start()

2019-07-05 14:36:34,410 root         INFO     Initializing block
2019-07-05 14:36:34,410 root         INFO     Initializing block
2019-07-05 14:36:34,413 root         INFO     Starting Block
2019-07-05 14:36:34,413 root         INFO     Starting Block
2019-07-05 14:36:34,416 root         INFO     Running trial {'iti': 5074, 'tr_num': 0, 'stim': '20190705-f00201-bos-calls-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}
2019-07-05 14:36:34,416 root         INFO     Running trial {'iti': 5074, 'tr_num': 0, 'stim': '20190705-f00201-bos-calls-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


{'iti': 5074, 'tr_num': 0, 'stim': '20190705-f00201-bos-calls-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


2019-07-05 14:36:44,216 root         INFO     Running trial {'iti': 6454, 'tr_num': 1, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}
2019-07-05 14:36:44,216 root         INFO     Running trial {'iti': 6454, 'tr_num': 1, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


{'iti': 6454, 'tr_num': 1, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


2019-07-05 14:36:55,387 root         INFO     Running trial {'iti': 3201, 'tr_num': 2, 'stim': '20190705-f00201-bos-calls-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}
2019-07-05 14:36:55,387 root         INFO     Running trial {'iti': 3201, 'tr_num': 2, 'stim': '20190705-f00201-bos-calls-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


...

2019-07-05 14:57:00,359 root         INFO     Running trial {'iti': 4870, 'tr_num': 118, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}
2019-07-05 14:57:00,359 root         INFO     Running trial {'iti': 4870, 'tr_num': 118, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


{'iti': 4870, 'tr_num': 118, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


2019-07-05 14:57:09,903 root         INFO     Running trial {'iti': 6790, 'tr_num': 119, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}
2019-07-05 14:57:09,903 root         INFO     Running trial {'iti': 6790, 'tr_num': 119, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


{'iti': 6790, 'tr_num': 119, 'stim': '20190701-f03123-bos-undirected_tag.wav', 'transparent_time': None, 'type': 'audio'}


2019-07-05 14:57:21,401 root         INFO     finished block
2019-07-05 14:57:21,401 root         INFO     finished block


### Leave the scheduler running

In [ ]:
block_runner.

In [13]:
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

### Debugging lines

In [18]:
pi_zmq.send_command(cmd_switch_glass('on'))

'cmd response: smartglass_switched on'

In [19]:
pi_zmq.send_command(cmd_switch_glass('off'))

'cmd response: smartglass_switched off'

In [6]:
pi_zmq.send_command(cmd_trial_number(5))

'cmd response: ok trial_number 5'

In [7]:
cmd_audio_trial('tone_test.wav')

b'trial_audio stim_file tone_test.wav'

In [7]:
rig_par['sbc'].send_command((cmd_audio_trial('bos_tag.wav')))

'cmd response: 0'

In [32]:
rig_par['sbc'].send_command((cmd_audio_trial(all_files[0])))

'cmd response: 0'